In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
import requests
import pandas as pd
import json
import talib
import numpy as np
import time

# from pandas_datareader import data as pdr
# import pandas_datareader.data as web
import yfinance as yf
import warnings
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings('ignore')


SECRET_FP = "./secrets.json"
with open(SECRET_FP, 'r') as file:
    secrets = json.load(file)
API_KEY = secrets['eodhd_api_key']



# Example list of US ETFs (add more as needed)
etf_tickers = ['VTI.US', 'SPY.US', 'VOO.US', 'IVV.US']

# Date range (adjust as needed)
start_date = '2023-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# Function to fetch historical data for one ETF
def fetch_etf_eod(ticker):
    url = f'https://eodhd.com/api/eod/{ticker}?from={start_date}&to={end_date}&api_token={API_KEY}&fmt=json'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if isinstance(data, list):
            df = pd.DataFrame(data)
            df['ticker'] = ticker
            return df
        else:
            print(f"Unexpected data format for {ticker}: {data}")
            return pd.DataFrame()
    else:
        print(f"Failed to fetch data for {ticker}. Status code: {response.status_code}")
        return pd.DataFrame()

# Collect data for all ETFs
all_etf_data = pd.concat([fetch_etf_eod(ticker) for ticker in etf_tickers], ignore_index=True)

# Save to CSV (optional)
# all_etf_data.to_csv('us_etf_historical_data.csv', index=False)
print("Saved historical data to 'us_etf_historical_data.csv'")


Saved historical data to 'us_etf_historical_data.csv'


In [2]:
all_etf_data

,date,open,high,low,close,adjusted_close,volume,ticker
0,2023-01-03,192.380,193.3600,188.930,190.41,183.7450,4092466,VTI.US
1,2023-01-04,191.510,193.0600,190.140,192.10,185.3758,3852482,VTI.US
2,2023-01-05,191.220,191.2700,189.460,189.85,183.2046,4149991,VTI.US
3,2023-01-06,191.120,194.6400,189.850,194.04,187.2479,3184261,VTI.US
4,2023-01-09,195.457,196.8882,194.008,194.12,187.3251,3888341,VTI.US
...,...,...,...,...,...,...,...,...
2679,2025-08-29,650.690,651.0600,646.350,648.32,648.3200,3709100,IVV.US
2680,2025-09-02,640.640,643.6400,638.050,643.42,643.4200,5796400,IVV.US
2681,2025-09-03,645.820,647.4000,643.640,646.88,646.8800,3886500,IVV.US
2682,2025-09-04,647.620,652.3600,646.700,652.21,652.2100,5855400,IVV.US


In [ ]:
import requests
import pandas as pd

def get_forward_returns(ticker: str, start_date: str, end_date: str, ndays: int):
    """
    Fetches historical EOD prices for a ticker and computes forward n-day returns.
    The forward period starts one day AFTER the current date.

    Args:
        ticker (str): Ticker symbol with exchange (e.g., 'AAPL.US').
        start_date (str): Start date in 'YYYY-MM-DD'.
        end_date (str): End date in 'YYYY-MM-DD'.
        ndays (int): Forward return horizon (number of days ahead).
        api_token (str): EODHD API token (default 'demo').

    Returns:
        pd.DataFrame with ['date', 'close', f'forward_{ndays}d_return'].
    """

    # --- Fetch EOD data ---
    url = f"https://eodhd.com/api/eod/{ticker}"
    params = {"from": start_date, "to": end_date, "api_token": API_KEY, "fmt": "json"}
    r = requests.get(url, params=params)
    data = r.json()

    # Convert to DataFrame
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)

    # --- Compute forward returns ---
    # Forward return = (P[t+n] / P[t+1]) - 1
    df[f'forward_{ndays}d_return'] = df['close'].shift(-ndays) / df['close'].shift(-1) - 1

    return df[['date', 'close', f'forward_{ndays}d_return']]


In [18]:
df_fwd_ret = get_forward_returns(
    ticker="IVV.US",
    start_date="2024-10-14",
    end_date="2025-09-05",
    ndays=20,
)
df_fwd_ret['date'] = df_fwd_ret['date'].astype(str)
print(df_fwd_ret.head(10))


         date   close  forward_20d_return
0  2024-10-14  587.15            0.032790
1  2024-10-15  582.49            0.025073
2  2024-10-16  585.08            0.025545
3  2024-10-17  585.25            0.014929
4  2024-10-18  587.46            0.003529
5  2024-10-21  586.59            0.008496
6  2024-10-22  586.13            0.021316
7  2024-10-23  580.79            0.019604
8  2024-10-24  582.03            0.025056
9  2024-10-25  581.90            0.025358


In [19]:
buy_signals.dtypes

tradetime                     object
call_vol                     float64
put_vol                      float64
pc_spread                    float64
pc_spread_rank_full          float64
pc_spread_percentile_full    float64
pc_spread_zscore_full        float64
dtype: object

In [20]:
buy_signals = pd.read_csv('./temp/pcs_rank_zscore_above_1.csv')
buy_signals

,tradetime,call_vol,put_vol,pc_spread,pc_spread_rank_full,pc_spread_percentile_full,pc_spread_zscore_full
0,2024-10-14,0.1436,0.2087,0.0651,0.766971,0.963333,1.690831
1,2024-11-14,0.1956,0.2456,0.0500,0.725132,0.930000,1.312747
2,2024-11-24,0.0944,0.1453,0.0509,0.727625,0.933333,1.335282
3,2024-11-26,0.0972,0.1367,0.0395,0.696038,0.893333,1.049840
4,2024-12-01,0.0785,0.1296,0.0511,0.728180,0.936667,1.340289
5,2024-12-11,0.0748,0.1429,0.0681,0.775284,0.970000,1.765948
6,2024-12-24,0.0798,0.1471,0.0673,0.773067,0.966667,1.745917
7,2025-01-12,0.1772,0.2493,0.0721,0.786367,0.973333,1.866102
8,2025-01-14,0.1276,0.2768,0.1492,1.000000,1.000000,3.796588
9,2025-01-26,0.1567,0.2022,0.0455,0.712663,0.906667,1.200072


In [21]:
buy_signals.merge(df_fwd_ret, left_on="tradetime", right_on="date", how="left")

,tradetime,call_vol,put_vol,pc_spread,pc_spread_rank_full,pc_spread_percentile_full,pc_spread_zscore_full,date,close,forward_20d_return
0,2024-10-14,0.1436,0.2087,0.0651,0.766971,0.963333,1.690831,2024-10-14,587.15,0.032790
1,2024-11-14,0.1956,0.2456,0.0500,0.725132,0.930000,1.312747,2024-11-14,596.23,0.031563
2,2024-11-24,0.0944,0.1453,0.0509,0.727625,0.933333,1.335282,NaN,NaN,NaN
3,2024-11-26,0.0972,0.1367,0.0395,0.696038,0.893333,1.049840,2024-11-26,603.76,0.003739
4,2024-12-01,0.0785,0.1296,0.0511,0.728180,0.936667,1.340289,NaN,NaN,NaN
5,2024-12-11,0.0748,0.1429,0.0681,0.775284,0.970000,1.765948,2024-12-11,610.59,-0.038149
6,2024-12-24,0.0798,0.1471,0.0673,0.773067,0.966667,1.745917,2024-12-24,604.36,-0.002814
7,2025-01-12,0.1772,0.2493,0.0721,0.786367,0.973333,1.866102,NaN,NaN,NaN
8,2025-01-14,0.1276,0.2768,0.1492,1.000000,1.000000,3.796588,2025-01-14,584.99,0.017763
9,2025-01-26,0.1567,0.2022,0.0455,0.712663,0.906667,1.200072,NaN,NaN,NaN
